In [151]:
import numpy as np
import pandas as pd
from pulp import *

In [2]:
pulp.pulpTestAll()

	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver <class 'pulp.solvers.PULP_CBC_CMD'> passed.
Solver <class 'pulp.solvers.CPLEX_DLL'> un

In [152]:
def MCLPModel(I, J, a, N, P):
    prob = LpProblem("MCLP", LpMaximize)
    x = LpVariable.dicts("x", J, lowBound = 0, upBound = 1, cat="Integer")
    y = LpVariable.dicts("y", I, lowBound = 0, upBound = 1, cat="Integer")
    
    # Maximizes
    prob += lpSum([a[i]*y[i] for i in I])
    
    # Constraints
    for i in I:
        prob += lpSum([x[j] for j in N[i]]) >= y[i]

    prob += lpSum([x[j] for j in J]) == P
    
    prob.solve()
    x_soln = np.array([x[j].varValue for j in J])
    facilities_ID = np.where(x_soln == 1)
    
    return [prob,facilities_ID]

In [155]:
def writeResult(result, prob, facilities, a, dataName, P):
    result.write("Data: " + dataName + "\n")
    result.write("Population Served is = %s\n" %value(prob.objective))
    result.write("The Ratio of covered population is = %s" %round(value(prob.objective)/sum(a) * 100, 1))
    result.write("% \n")
    result.write("location = %s\n\n" %facilities[0])

In [153]:
# variable setting
I = [i for i in range(5915)] # demand nodes
J = [j for j in range(1991)] # facility sites
S = 200                       # maximum distance
dist = np.loadtxt("C:/smoking/data/analysis/distanceMatrix.csv", delimiter=",", dtype=np.float32) # distance matrix
N = [[j for j in J if dist[i][j] < S] for i in I] # facility sites that can be reached from demand nodes within maximum distance

In [ ]:
time = ["morning", "noon", "afternoon", "evening", "night"] 
week = ["weekday", "weekend"]
Plist = [1,5,10,15,20,30,40,60,80,100,120,150,200]
# analysis result file
result = open("C:/smoking/result.txt","w")

for w in week:
    for t in time:
        for P in Plist:
            dataName = "_".join([t,w]) + ".csv"
            A = pd.read_csv("C:/smoking/data/analysis/"+dataName)
            a = A.SMOKER
            
            prob, facilities = MCLPModel(I, J, a, N, P)
            writeResult(result, prob, facilities, a, dataName, P)         
            
            print(dataName, P, "complete")

result.close()


morning_weekday.csv 1 complete
morning_weekday.csv 5 complete
morning_weekday.csv 10 complete
morning_weekday.csv 15 complete
morning_weekday.csv 20 complete
morning_weekday.csv 30 complete
morning_weekday.csv 40 complete
morning_weekday.csv 60 complete
morning_weekday.csv 80 complete
morning_weekday.csv 100 complete
morning_weekday.csv 120 complete
morning_weekday.csv 150 complete
morning_weekday.csv 200 complete
noon_weekday.csv 1 complete
noon_weekday.csv 5 complete
noon_weekday.csv 10 complete
noon_weekday.csv 15 complete
noon_weekday.csv 20 complete
noon_weekday.csv 30 complete
noon_weekday.csv 40 complete
noon_weekday.csv 60 complete
